#CODE'S EXPLANATION


Firstly, we have created a DataFrame with all the search links for Airbnb, following search factors such as location and price. This will serve us to perform the scraping and obtain all the rooms' URLs. In our case, we focus on the Andalucia region (Spain).

In a second scraping, we will gather the information that appears within each of the rooms' links. To do this, we will utilize the scraping links obtained previously. At this point, we will obtein the links to the profiles of the hosts of each room's link, which will be used for the next step.

Thirdly, thanks to the links to the hosts' profiles, we will perform scraping of the information found within the host profiles.

Finally, we will export all the information to MySQL and establish Primary Keys and Foreign Keys for each DataFrame.


In [ ]:
#LIBRARY

from time import sleep
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re
from sqlalchemy import create_engine
import mysql.connector

In [ ]:
#CREATE WEBDRIVER

opts = Options()
opts.add_argument("user-agent=Chrome/115.0.5790.102")
opts.add_argument("--headless")
opts.add_argument("--disable-gpu")  # Disabling GPU in headless mode.
opts.add_argument("--window-size=1920x1080")  # Window size in headless mode.

driver = webdriver.Chrome(
    service=Service(executable_path="C:/ / / / /binman/binman_chromedriver/win32/115.0.5790.102/chromedriver.exe"),
    options=opts
)

In [ ]:
#COOKIES

driver.get("https://www.airbnb.com/")
time.sleep(5)

node = "/html/body/div[5]/div/div/div[1]/div/div[4]/section/div[2]/div[2]/button"

driver.find_element(By.XPATH, node).click()

In [ ]:
#CREATE SEARCH RANGE

url_range_min = list(range(1, 1001, 10))
url_range_max = list(range(10, 1010, 10))

#You should include all the key locations you want to gather information about.
localization_list = ["Granada", "Malaga", "Almeria", "Jaen", "Cordoba", "Sevilla", "Cadiz", "Huelva", 
                     "Motril", "Baza", "Marbella", "Antequera", "Roquetas-de-Mar", "Carboneras", "Cazorla",
                     "Lucena", "Arcos-de-la-Frontera", "Tarifa", "Aracena"] 

urls = []

for localization in localizacion_list:
    for min_val, max_val in zip(url_range_min, url_range_max):
        url = f"https://www.airbnb.com/s/{localization}/homes?date_picker_type=flexible_dates&price_max={max_val}&price_min={min_val}"
        urls.append(url)
        
special_url = [f"https://www.airbnb.com/s/{localization}/homes?date_picker_type=flexible_dates&price_min=1000" for localization in localization_list]

urls = urls + special_url

data = {"url": urls}
df = pd.DataFrame(data)

print(df)

In [ ]:
#FUNCTION FOR FIRTS SCRAPING: LINK

def extraer_link(url):
    
    df_link = pd.DataFrame(columns=["link", "url_general", "id_room"])

    try:
        driver.get(url)
        time.sleep(5)
        
        data = []
        elements = driver.find_elements(By.CSS_SELECTOR, "a.l1ovpqvx.bn2bl2p.dir.dir-ltr")
        for element in elements:
            data.append({"link": element.get_attribute("href"),
                         "url_general": url,
                         "id_room": int(re.search(r'/rooms/(\d+)', element.get_attribute("href")).group(1))})
        
        df_link = pd.concat([df_link, pd.DataFrame(data)])
        
        try:
            vermas = driver.find_element(By.CSS_SELECTOR, ".l1ovpqvx.c1ytbx3a.dir.dir-ltr")
            while vermas.is_displayed():
                vermas.click()
                time.sleep(5)
                
                data = []
                elements = driver.find_elements(By.CSS_SELECTOR, "a.l1ovpqvx.bn2bl2p.dir.dir-ltr")
                for element in elements:
                    data.append({"link": element.get_attribute("href"),
                                 "url_general": url,
                                 "id_room": int(re.search(r'/rooms/(\d+)', element.get_attribute("href")).group(1))})
                
                df_link = pd.concat([df_link, pd.DataFrame(data)])
                time.sleep(5)
                
        except Exception as e:
            pass
        
        print("Extrajiste de:", url)
        time.sleep(5)
    
    except Exception as e:
        pass
    
    return df_link

In [ ]:
#FIRTS SCRAPING: lINK

df_link = pd.DataFrame()

for url in df['url']:
    extracted_data = extraer_link(url)
    df_link = pd.concat([df_link, extracted_data], ignore_index=True)
    

In [ ]:
#DELETE NaN AND DUPLICATE

df_link = df_link.dropna(subset=['id_room'])

df_link = df_link.drop_duplicates(subset=['id_room'], ignore_index=True)

df_link

In [ ]:
#SAVE DATAFRAME

df_link.to_csv("C:/Users/Pc/Desktop/df_link.csv", index=False)

In [ ]:
#FUNCTION FOR SECOND SCRAPING: ROOM

def extraer_datos(url):
    
    df_room = pd.DataFrame(columns = ["titulo",
                                      "localizacion",
                                      "tipo_alojamiento",
                                      "capacidad",
                                      "latitud",
                                      "longitud",
                                      "reviews",
                                      "puntuacion",
                                      "precio",
                                      "id_room",
                                      "id_host",
                                      "perfil_host",
                                      "url",
                                      "fecha_extraccion"
                                     ])
     
    try:
        
    #Access the website and scroll down to the map.
    
        driver.get(url)
        
        time.sleep(5)
        
        selector = "#site-content > div > div:nth-child(1) > div:nth-child(5) > div > div > div > div:nth-child(2) > section > div._384m8u"

        script = f'''
            var element = document.querySelector('{selector}');
            if (element) {{
                element.scrollIntoView({{ behavior: 'smooth' }});
            }}
        '''

        driver.execute_script(script)
        
        time.sleep(3)
        
    #NODES
        
        nodo_titulo = "._1n81at5"
    
        nodo_localizacion = "._9xiloll"

        nodo_tipo_alojamiento = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/div/div/div[1]/section/h2"

        nodo_tipo_alojamiento2 = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/section/div/div/div[1]/div/h2"

        nodo_capacidad = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/div/ul/li[1]/div[2]/div"

        nodo_capacidad2 = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[1]/div/div[1]/div/div/section/div/div/div[1]/ol/li[1]/span[1]"

        nodo_reviews = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[4]/div/div/div/div[2]/section/div[1]/span[2]/h2/div/span"

        nodo_reviews2 = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div/div/div[1]/div[2]/span/span/button/span"

        nodo_precio = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[2]/div/div/div[1]/div/div/div/div/div/div/div/div[1]/div[1]/div/div/span/div/span[1]"
        
        nodo_coordenadas = "#site-content > div > div:nth-child(1) > div:nth-child(5) > div > div > div > div:nth-child(2) > section > div._384m8u > div:nth-child(4) > div > div > div > div:nth-child(15) > div > a"

        nodo_host = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[6]/div/div/div/div[2]/div/section/div[1]/div[1]/div/a"

        nodo_host2 = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[1]/div/div[4]/div/div[2]/section/div[2]/div/div/div[1]/div[2]/a"

        nodo_host3 = "/html/body/div[5]/div/div/div[1]/div/div[2]/div/div/div/div[1]/main/div/div[1]/div[3]/div/div[1]/div/div[3]/div/div[2]/section/div[2]/div/div/div[1]/div[2]/a"
        
    #TITULO
        
        try:
            titulo = driver.find_element(By.CSS_SELECTOR, nodo_titulo).text
        except NoSuchElementException:
            titulo = None
            
        
    #LOCALIZACION     
        
        try:
            localizacion = driver.find_element(By.CSS_SELECTOR, nodo_localizacion).text
        except NoSuchElementException:
            titulo = None
        
    #TIPO_ALOJAMIENTO        
        
        try:
            tipo_alojamiento = driver.find_element(By.XPATH, nodo_tipo_alojamiento).text
            tipo_alojamiento = re.sub("\\..*", "", tipo_alojamiento)

        except NoSuchElementException:
            try:
                tipo_alojamiento = driver.find_element(By.XPATH, nodo_tipo_alojamiento2).text
                tipo_alojamiento = re.sub("\\..*", "", tipo_alojamiento)

            except NoSuchElementException:
                tipo_alojamiento = None
        
    #CAPACIDAD
        
        try:
            capacidad = driver.find_element(By.XPATH, nodo_capacidad).text
            capacidad = int(re.sub("\\D", "", capacidad))

        except NoSuchElementException:
            try:
                capacidad = driver.find_element(By.XPATH, nodo_capacidad2).text
                capacidad = int(re.sub("\\D", "", capacidad))

            except NoSuchElementException:
                capacidad = None 
        
    #REVIEWS
        
        try:
            reviews = driver.find_element(By.XPATH, nodo_reviews).text
            reviews = re.search(r'·\s*(\d+)\s+evaluaciones', reviews).group(1)
            reviews = re.sub(r'\D', '', reviews) 
            reviews = int(reviews) 

        except NoSuchElementException:
            try:
                reviews = driver.find_element(By.XPATH, nodo_reviews2).text
                reviews = re.sub(r'\D', '', reviews) 
                reviews = int(reviews) 

            except NoSuchElementException:
                reviews = 0 
                
    #PUNTUACION
        
        try:
            puntuacion = driver.find_element(By.XPATH, nodo_reviews).text
            puntuacion = float(re.search(r'\d+(,\d+)?', puntuacion).group().replace(',', '.'))

        except NoSuchElementException:
            try:
                puntuacion = driver.find_element(By.XPATH, nodo_reviews).text
                puntuacion = float(re.search(r'\d+(,\d+)?', puntuacion).group().replace(',', '.'))

            except NoSuchElementException:
                puntuacion = None 
                
    #PRECIO
        
        try:
            precio = driver.find_element(By.XPATH, nodo_precio).text
            precio = int(re.sub("\\D", "", precio))

        except NoSuchElementException:
            precio = None 
                
    #COORDENADAS
        
        try:
            coordenadas = re.search(r'll=(-?\d+\.\d+),(-?\d+\.\d+)', driver.find_element(By.CSS_SELECTOR, nodo_coordenadas).get_attribute("href"))
            latitud = float(coordenadas.group(1))
            longitud = float(coordenadas.group(2))

        except NoSuchElementException:
            latitud = None
            longitud = None
            
    #PERFIL HOST

        try:
            perfil_host = driver.find_element(By.XPATH, nodo_host).get_attribute("href")

        except NoSuchElementException:
            try:
                perfil_host = driver.find_element(By.XPATH, nodo_host2).get_attribute("href")

            except NoSuchElementException:
                try:
                    perfil_host = driver.find_element(By.XPATH, nodo_host3).get_attribute("href")

                except NoSuchElementException:
                    perfil_host = None 
                    
    #ID_ROOM (PRIMARY KEY)
    
        id_room = re.search(r'/rooms/(\d+)', url).group(1)
        id_room = int(id_room)
    
    #ID_HOST (FOREIGN KEY)
    
        try:    
            id_host = re.search(r'\d+$', perfil_host).group()
            id_host = int(id_host)
            
        except NoSuchElementException:
            id_host = None
                    
    #URL
        
        url = re.sub("\\?.*", "", url)
        
    #FECHA_EXTRACCION
        
        fecha_extraccion = pd.Timestamp.now().date()
              
    #MERGE THE INFORMATION
        
        data = {
            "titulo": titulo,
            "localizacion": localizacion,
            "tipo_alojamiento": tipo_alojamiento,
            "capacidad": capacidad,
            "latitud": latitud,
            "longitud": longitud,
            "reviews": reviews,
            "puntuacion": puntuacion,
            "precio": precio,
            "id_room": id_room,
            "id_host": id_host,
            "perfil_host": perfil_host,
            "url": url,
            "fecha_extraccion": fecha_extraccion
        }
        
        df_room = pd.DataFrame(data, index=[0])
        
    
    except Exception as e:
        pass
    
    return df_room

In [ ]:
#BLOCK TRANSLATION 

url = "https://www.airbnb.es/rooms/40911785?adults=1&category_tag=Tag%3A8678&children=0&enable_m3_private_room=true&infants=0&pets=0&photo_id=1169475226&check_in=2023-08-22&check_out=2023-08-27&source_impression_id=p3_1692525090_MxeN582L%2FoLyJz4l&previous_page_section_name=1000&federated_search_id=486da6af-b718-4875-bba7-4bf5001e01bc&_set_bev_on_new_domain=1692534067_OThiOTk3ZjVhNTgy"

driver.get(url)

time.sleep(7)

driver.find_element(By.XPATH, "/html/body/div[9]/div/section/div/div/div[2]/div/div[2]/div/div[4]/button").click()

time.sleep(3)

driver.find_element(By.XPATH, "/html/body/div[10]/div/section/div/div/div[2]/div/div[2]/section[1]/div/ul/li/div/div[2]/button").click()


In [ ]:
#SECOND SCRAPING: ROOM

df_room = pd.DataFrame()

counter = 0
total = len(df_link['link'])

for url in df_link['link']:
    extracted_data = extraer_datos(url)
    df_room = pd.concat([df_room, extracted_data], ignore_index=True)
    
    counter += 1
    print(f"Ya extrajiste de la pag: {counter} / {total} ({round(counter/total*100, 1)}%)")

In [ ]:
#DELETE NaN AND DUPLICATE

df_room = df_room.dropna(subset=['id_room'])

df_room = df_room.drop_duplicates(subset=['id_room'], ignore_index=True)

df_room

In [ ]:
#FUNCTION FOR THIRD SCRAPING: HOST

def extraer_host(url):
    
    df_host = pd.DataFrame(columns = ["nombre",
                                      "evaluaciones",
                                      "valoracion",
                                      "experiencia_años",
                                      "anuncios",
                                      "id_host",
                                      "url",
                                      "fecha_extraccion"])
    
    try:
        
    #ACCESS THE WEBSITE
    
        driver.get(url)
        
        time.sleep(3)
        
    #NODES
        
        nodo_nombre = "/html/body/div[5]/div/div/div[1]/div/div[2]/div[1]/main/div/div/section/div/div[1]/div[1]/div/div/section/div[1]/div[2]/h1/div/span"       
        nodo_evaluaciones = "/html/body/div[5]/div/div/div[1]/div/div[2]/div[1]/main/div/div/section/div/div[1]/div[1]/div/div/section/div[2]/div[1]/div/span[1]"
        nodo_valoracion = "/html/body/div[5]/div/div/div[1]/div/div[2]/div[1]/main/div/div/section/div/div[1]/div[1]/div/div/section/div[2]/div[2]/div/span[1]/div"       
        nodo_experiencia = "/html/body/div[5]/div/div/div[1]/div/div[2]/div[1]/main/div/div/section/div/div[1]/div[1]/div/div/section/div[2]/div[3]/div/span[1]"
        nodo_condicion = "/html/body/div[5]/div/div/div[1]/div/div[2]/div[1]/main/div/div/section/div/div[1]/div[1]/div/div/section/div[2]/div[3]/div/span[2]"
        nodo_anuncios = "/html/body/div[5]/div/div/div[1]/div/div[2]/div[1]/main/div/div/section/div/div[2]/div/div/div[7]/div/section/div[2]/div[1]"
        nodo_anuncios2 = "/html/body/div[5]/div/div/div[1]/div/div[2]/div[1]/main/div/div/section/div/div[2]/div/div/div[6]/div/section/div[2]/div[1]"

    #NOMBRE
        
        try:
            nombre = driver.find_element(By.XPATH, nodo_nombre).text

        except NoSuchElementException:
            nombre = None
        
    #EVALUACIONES
        
        try:
            evaluaciones = driver.find_element(By.XPATH, nodo_evaluaciones).text
            evaluaciones = int(evaluaciones)
        
        except NoSuchElementException:
            evaluaciones = None
        
    #VALORACION
        
        try:
            valoracion = driver.find_element(By.XPATH, nodo_valoracion).text
            valoracion = float(re.sub(",", ".", valoracion))

        except NoSuchElementException:
            valoracion = None
        
    #EXPERIENCIA
        
        try:    
            condicion = driver.find_element(By.XPATH, nodo_condicion).text
            condicion = int(condicion) #Ademas de descargar la condicion, creo un "ValueError" intencionado que provoque el primer Except

        except ValueError as e:
            if "Año" in str(condicion):
                experiencia_años = driver.find_element(By.XPATH, nodo_experiencia).text
                experiencia_años = int(experiencia_años)

            else:
                experiencia_años = 0

        except NoSuchElementException:
            experiencia_años = 0
        
    #ANUNCIOS
    
        try:    
            anuncios = driver.find_element(By.XPATH, nodo_anuncios).text
            anuncios = re.search(r'\((.*?)\)', anuncios).group(1)
            anuncios = int(re.sub(r'\D', '', anuncios))
            
        except NoSuchElementException:
            try:
                anuncios = driver.find_element(By.XPATH, nodo_anuncios2).text
                anuncios = re.search(r'\((.*?)\)', anuncios).group(1)
                anuncios = int(re.sub(r'\D', '', anuncios))
                
            except NoSuchElementException:              
                anuncios = None
            
    #ID_HOST (PRIMARY KEY)
    
        id_host = re.search(r'\d+$', url).group()
        id_host = int(id_host)
        
    #URL
    
        url = url
    
    #FECHA_EXTRACCION
        
        fecha_extraccion = pd.Timestamp.now().date()
    
    #MERGE THE INFORMATION
        
        data = {
            "nombre": nombre,
            "evaluaciones": evaluaciones,
            "valoracion": valoracion,
            "experiencia_años": experiencia_años,
            "anuncios": anuncios,
            "id_host": id_host,
            "url": url,
            "fecha_extraccion": fecha_extraccion
        }


        df_host = pd.DataFrame(data, index=[0])
        
    
    except Exception as e:
        pass
    
    return df_host

In [ ]:
#THIRD SCRAPING: HOST

df_host = pd.DataFrame()

counter = 0
total = len(df_room['perfil_host'])

for url in df_room['perfil_host']:
    extracted_data = extraer_host(url)
    df_host = pd.concat([df_host, extracted_data], ignore_index=True)
    
    counter += 1
    print(f"Ya extrajiste de la pag: {counter} / {total} ({round(counter/total*100, 1)}%)")


In [ ]:
#DELETE NaN AND DUPLICATE

df_host = df_host.dropna(subset=['id_host'])

df_host = df_host.drop_duplicates(subset=['id_host'], ignore_index=True)

df_host

In [ ]:
#SAVE DATAFRAME

df_host.to_csv("C:/Users/Pc/Desktop/df_host.csv", index=False)

In [ ]:
# CLOSE THE WEBDRIVER

driver.quit()

In [ ]:
#EXPORT DATAFRAMES TO MYSQL

#Conexion

usuario = ""
contraseña = ""
host = ""
puerto =   
base_de_datos = ""

url_conexion = f"mysql+pymysql://{usuario}:{contraseña}@{host}:{puerto}/{base_de_datos}"

# Create engine
engine = create_engine(url_conexion)

# Export DataFrames
df_link.to_sql("link", con=engine, if_exists="replace", index=False)
df_room.to_sql("room", con=engine, if_exists="replace", index=False)
df_host.to_sql("host", con=engine, if_exists="replace", index=False)


In [ ]:
#CREATE PRIMARY KEYS AND FOREIGN KEYS

#Configuration of Conexion
config = {
    "user": "",
    "password": "",
    "host": "",
    "database": ""
}

# Conexion
conn = mysql.connector.connect(**config)
cursor = conn.cursor()

#PRIMARY KEYS
    #Df_link
sql_query = "ALTER TABLE airbnb.link ADD PRIMARY KEY (link(255));"
cursor.execute(sql_query)
    #Df_room
sql_query = "ALTER TABLE airbnb.room ADD PRIMARY KEY (id_room);"
cursor.execute(sql_query)
    #Df_host
sql_query = "ALTER TABLE airbnb.host ADD PRIMARY KEY (id_host);"
cursor.execute(sql_query)
    
#FOREIGN KEYS
    #Df_link
sql_query = "ALTER TABLE airbnb.link ADD FOREIGN KEY (id_room) REFERENCES room(id_room);"
cursor.execute(sql_query) 
    #Df_room
sql_query = "ALTER TABLE airbnb.room ADD FOREIGN KEY (id_host) REFERENCES host(id_host);"
cursor.execute(sql_query)    

# Confirm the changes and close the connection
conn.commit()
cursor.close()
conn.close()
